<a href="https://colab.research.google.com/github/yonseimath/datascience-biginner-2022-kaggle-competitions/blob/feature%2Fyenakim/yenakim/AI4Code_TF_TPU_CodeBert_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/nickuzmenkov/ai4code-tf-tpu-codebert-data-preparation/notebook의 코드를 공부한 노트북입니다.

# Setup

In [ ]:
!mkdir 'raw' 'tfrec'

In [ ]:
import glob
import json
import os
from typing import List

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.model_selection import GroupKFold
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

In [ ]:
RANDOM_STATE = 42
MD_MAX_LEN = 64 # 마크다운 토큰은 최대 64 토큰까지
TOTAL_MAX_LEN = 512 # Code context는 512 토큰까지(23 토큰 이하로 구성된 코드 셀 20개까지)
K_FOLDS = 5
FILES_PER_FOLD = 16
LIMIT = 1_000 if os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive" else None # KAGGLE_KERNEL_RUN_TYPE 환경 변수가 Interactive이면 앞의 1000개의 노트북만을사용한다
# LIMIT = None # 전체 노트북 사용
MODEL_NAME = "microsoft/codebert-base"
TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
INPUT_PATH = "../input/AI4Code"

In [ ]:
# 노트북의 경로를 변수로 받아 내용을 읽어온 후 DataFrame 형식으로 return
def read_notebook(path: str) -> pd.DataFrame:
    return (
        pd.read_json(path, dtype={"cell_type": "category", "source": "str"})
        .assign(id=os.path.basename(path).split(".")[0])
        .rename_axis("cell_id")
    ) 

# 셀에 있는 \n 코드가 \\n으로 입력되므로 replace 해주고 이를 다시 return
def clean_code(cell: str) -> str:
    return str(cell).replace("\\n", "\n")

# n보다 cell들이 많으면 샘플을 뽑고, 그렇지 않으면 그대로 cells를 return
def sample_cells(cells: List[str], n: int) -> List[str]:
   cells = [clean_code(cell) for cell in cells] 
    if n >= len(cells):
        return cells
    else:
        results = []
        step = len(cells) / n
        idx = 0
        while int(np.round(idx)) < len(cells): # 전체 샘플 중 step만큼 건너뛴 샘플들을 골라 result에 넣음
            results.append(cells[int(np.round(idx))])
            idx += step
        if cells[-1] not in results:
            results[-1] = cells[-1] # 마지막 셀은 반드시 넣음
        return results

# total_code(전체 코드), total_md(전체 마크다운), codes(코드에서 샘플 추출) 특성 생성
def get_features(df: pd.DataFrame) -> dict:
    features = {}
    for i, sub_df in tqdm(df.groupby("id"), desc="Features"):
        features[i] = {}
        total_md = sub_df[sub_df.cell_type == "markdown"].shape[0]
        code_sub_df = sub_df[sub_df.cell_type == "code"]
        total_code = code_sub_df.shape[0]
        codes = sample_cells(code_sub_df.source.values, 20)
        features[i]["total_code"] = total_code
        features[i]["total_md"] = total_md
        features[i]["codes"] = codes
    return features

# input 값을 준비
def tokenize(df: pd.DataFrame, fts: dict) -> dict:
    input_ids = np.zeros((len(df), TOTAL_MAX_LEN), dtype=np.int32)
    attention_mask = np.zeros((len(df), TOTAL_MAX_LEN), dtype=np.int32)
    features = np.zeros((len(df),), dtype=np.float32)
    labels = np.zeros((len(df),), dtype=np.float32)

    for i, row in tqdm( # tqdm : 진행률을 보여주는 바 생성
        df.reset_index(drop=True).iterrows(), desc="Tokens", total=len(df) 
    ): # iterrows : 행번호와 값 동시에 출력, reset_index : 인덱스 초기화
        row_fts = fts[row.id]

        inputs = TOKENIZER.encode_plus(
            row.source,
            None,
            add_special_tokens=True, # 토큰의 시작점에 [CLS] 토큰, 토큰의 마지막에 [SEP] 토큰을 붙인다
            max_length=MD_MAX_LEN,
            padding="max_length",
            return_token_type_ids=True, # token type id 생성(0과 1로 문장의 토큰 값 분리)
            truncation=True,
        ) # Markdown(~64)
        code_inputs = TOKENIZER.batch_encode_plus(
            [str(x) for x in row_fts["codes"]] or [""],
            add_special_tokens=True,
            max_length=23,
            padding="max_length",
            truncation=True,
        ) # code context(~512)

        ids = inputs["input_ids"] # 토크나이즈 된 MD
        for x in code_inputs["input_ids"]:
            ids.extend(x[:-1]) # 토크나이즈 된 코드를 ids 뒤에 이어붙임
        ids = ids[:TOTAL_MAX_LEN] # 512 토큰까지만 사용
        if len(ids) != TOTAL_MAX_LEN: # 토큰의 길이가 512보다 작으면 그만큼 pad_token_id를 이어붙임
            ids = ids + [
                TOKENIZER.pad_token_id,
            ] * (TOTAL_MAX_LEN - len(ids))

        mask = inputs["attention_mask"] # 패딩된 부분이 학습에 영향을 주지 않도록 처리하는 입력값, 1은 어텐션에 영향을 받는 토큰이고 0은 영향을 받지 않는 토큰
        for x in code_inputs["attention_mask"]:
            mask.extend(x[:-1]) # 코드 마스크도 뒤에 이어붙임
        mask = mask[:TOTAL_MAX_LEN] # 512 토큰까지만 사용
        if len(mask) != TOTAL_MAX_LEN: # 토큰의 길이가 512보다 작으면 그만큼 pad_token_id를 이어붙임
            mask = mask + [
                TOKENIZER.pad_token_id,
            ] * (TOTAL_MAX_LEN - len(mask))

        input_ids[i] = ids # 결과 배열에 넣음
        attention_mask[i] = mask
        features[i] = (
            row_fts["total_md"] / (row_fts["total_md"] + row_fts["total_code"]) or 1 # 분모가 1일 때 예외처리?
        ) # 특성은 MD 셀 수 / 전체 셀 수
        labels[i] = row.pct_rank # 레이블은 자료의 순서(rank)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "features": features,
        "labels": labels,
    }

# 순서를 가져옴
def get_ranks(base: pd.Series, derived: List[str]) -> List[str]:
    return [base.index(d) for d in derived]

# 직렬화? 데이터 포맷을 바꿈
def _serialize_sample(
    input_ids: np.array,
    attention_mask: np.array,
    feature: np.float64,
    label: np.float64,
) -> bytes:
    feature = {
        "input_ids": tf.train.Feature(int64_list=tf.train.Int64List(value=input_ids)),
        "attention_mask": tf.train.Feature(
            int64_list=tf.train.Int64List(value=attention_mask)
        ),
        "feature": tf.train.Feature(float_list=tf.train.FloatList(value=[feature])),
        "label": tf.train.Feature(float_list=tf.train.FloatList(value=[label])),
    }
    sample = tf.train.Example(features=tf.train.Features(feature=feature))
    return sample.SerializeToString()


def serialize(
    input_ids: np.array,
    attention_mask: np.array,
    features: np.array,
    labels: np.array,
    path: str,
) -> None:
    with tf.io.TFRecordWriter(path) as writer:
        for args in zip(input_ids, attention_mask, features, labels):
            writer.write(_serialize_sample(*args))

# Collect Data

In [ ]:
paths = glob.glob(os.path.join(INPUT_PATH, "train", "*.json"))
if LIMIT is not None:
    paths = paths[:LIMIT]
# 노트북 읽어옴
df = (
    pd.concat([read_notebook(x) for x in tqdm(paths, desc="Concat")])
    .set_index("id", append=True) # id열을 index로 지정
    .swaplevel() # Multi Index에서 두 인덱스의 순서를 변경하는 메서드
    .sort_index(level="id", sort_remaining=False) # id를 기준으로 정렬
)

# 셀 순서 읽어옴
df_orders = pd.read_csv(
    os.path.join(INPUT_PATH, "train_orders.csv"),
    index_col="id",
    squeeze=True,
).str.split()
df_orders_ = df_orders.to_frame().join(
    df.reset_index("cell_id").groupby("id")["cell_id"].apply(list),
    how="right",
)

# 셀 순서 가져옴
ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {"cell_id": cell_id, "rank": get_ranks(cell_order, cell_id)}
df_ranks = (
    pd.DataFrame.from_dict(ranks, orient="index") # 딕셔너리를 입력받아 DataFrame을 반환, orient = index로 설정하면 딕셔너리의 키를 행의 레이블로 설정
    .rename_axis("id")
    .apply(pd.Series.explode)
    .set_index("cell_id", append=True)
)

# 상위가 있는(fork된) 노트북 불러옴
df_ancestors = pd.read_csv(
    os.path.join(INPUT_PATH, "train_ancestors.csv"), index_col="id"
)
df = (
    df.reset_index()
    .merge(df_ranks, on=["id", "cell_id"])
    .merge(df_ancestors, on=["id"])
) # df에 각 노트북 기준으로 상위 노트북을 merge하고, 셀 기준으로 각 셀의 순서를 merge함

df["pct_rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count") # 퍼센트 랭크, 1/5(다섯 중 첫번째)와 같은 표현
df = df.sort_values("pct_rank").reset_index(drop=True) # 퍼센트 랭크를 기준으로 정렬, drop=True : 기존 인덱스가 첫번째 열로 자동 삽입되지 않도록

features = get_features(df) # 특성 생성

df = df[df["cell_type"] == "markdown"] # df에는 cell type이 markdown인 자료만 있음
df = df.drop(["rank", "parent_id", "cell_type"], axis=1).dropna() # rank, paren_id, cell_type 행을 drop

# Make Tokens & Save

In [ ]:
df.to_csv("data.csv") # markdown 데이터프레임 csv 파일로 저장
with open("features.json", "w") as file:
    json.dump(features, file) # features.json 파일에 생성된 특성 저장(코드와 MD 모두)

In [ ]:
df = shuffle(df, random_state=RANDOM_STATE) # MD 셀 순서 섞음

# 폴드 디렉토리 연결
for fold, (_, split) in enumerate( # (인덱스, 원소)
    GroupKFold(K_FOLDS).split(df, groups=df["ancestor_id"]) # ancestor_id에 기반한 그룹별 교차 검증, 각 분할에서 한 그룹 전체가 훈련 세트 또는 테스트 세트에 속함
):
    print("=" * 36, f"Fold {fold}", "=" * 36)
    fold_dir = f"tfrec/{fold}"
    if not os.path.exists(fold_dir):
        os.mkdir(fold_dir)

    data = tokenize(df.iloc[split], features)

# 압축된 .npz 형식의 여러 파일을 단일 파일로 저장
    np.savez_compressed(
        f"raw/{fold}.npz",
        input_ids=data["input_ids"],
        attention_mask=data["attention_mask"],
        features=data["features"],
        labels=data["labels"],
    )

# 직렬화 후 파일 저장
    for split, index in tqdm(
        enumerate(np.array_split(np.arange(data["labels"].shape[0]), FILES_PER_FOLD)),
        desc=f"Saving",
        total=FILES_PER_FOLD,
    ):
        serialize(
            input_ids=data["input_ids"][index],
            attention_mask=data["attention_mask"][index],
            features=data["features"][index],
            labels=data["labels"][index],
            path=os.path.join(fold_dir, f"{split:02d}-{len(index):06d}.tfrec"),
        )